In [212]:
import pdfplumber
import pandas as pd


def extract_pdf_text(pdf_path):
    # Open the PDF file
    with pdfplumber.open(pdf_path) as pdf:
        # Initialize an empty list to store the extracted data
        data = []
        # Iterate through each page
        for page_num, page in enumerate(pdf.pages):
            # Extract the text with bounding boxes
            for element in page.extract_words():
                text = element['text']
                x0, y0, x1, y1 = element['x0'], element['top'], element['x1'], element['bottom']
                data.append([page_num + 1, text, x0, y0, x1, y1])

        # Convert the list to a DataFrame
        df = pd.DataFrame(data, columns=['page', 'text', 'x0', 'y0', 'x1', 'y1'])
        return df


def find_interval(number, intervals):
    intervals = sorted(intervals)
    for interval in intervals:
        if number >= interval[0] and number < interval[1]:
            return interval
    return None


def assign_intervals_and_values(df, gridlines):
    df["interval"] = df['x0'].apply(lambda x: find_interval(x, list(gridlines.keys())))
    df["value"] = df["interval"].apply(lambda x: gridlines.get(x, None) if x else None)
    return df


def process_consecutive_values(df, target_value):
    processed_rows = []
    current_row = None

    for _, row in df.iterrows():
        if row['value'] == target_value:
            if current_row is None:
                current_row = row.copy()
            else:
                # Concatenate the text field
                current_row['text'] += row['text']
                # Update the bounding box
                current_row['x1'] = max(current_row['x1'], row['x1'])
                current_row['y1'] = max(current_row['y1'], row['y1'])
        else:
            if current_row is not None:
                processed_rows.append(current_row)
                current_row = None
            processed_rows.append(row)

    if current_row is not None:
        processed_rows.append(current_row)

    return pd.DataFrame(processed_rows)


In [213]:
def update_identifier_ranges(df, unique_identifier):
    df_identifier = df[df["value"] == unique_identifier].sort_values(by=["page","y1"]).reset_index(drop=True)
    for index in df_identifier.index:
        if index == 0:
            df_identifier.loc[index, "identifier_range_y0"] = df_identifier.loc[index, 'y0'] 
            df_identifier.loc[index, "identifier_range_y1"] = df_identifier.loc[index, 'y1']
        elif index == df_identifier.index.max():
            df_identifier.loc[index, "identifier_range_y0"] = df_identifier.loc[index-1, 'y1'] 
            df_identifier.loc[index, "identifier_range_y1"] = df_identifier.loc[index, 'y1']
        else:
            df_identifier.loc[index, "identifier_range_y0"] = df_identifier.loc[index-1, 'y1']
            df_identifier.loc[index, "identifier_range_y1"] = df_identifier.loc[index, 'y1']

    df_identifier = df_identifier[["page", "text", "identifier_range_y0", "identifier_range_y1"]]
    return df_identifier

In [214]:
def concatenate_values(df):
    new_value_column = []
    new_text_column = []

    # Iterate over the dataframe to concatenate values
    current_value = None
    current_text = ""

    for value, text in zip(df['value'], df['text']):
        if value == current_value:
            current_text += text + " "
        else:
            if current_value is not None:
                new_value_column.append(current_value)
                new_text_column.append(current_text.strip())
            current_value = value
            current_text = text + " "  # Added space at the end of each text

    # Append the last accumulated values
    if current_value is not None:
        new_value_column.append(current_value)
        new_text_column.append(current_text.strip())

    # Create a new DataFrame with the concatenated values
    new_df = pd.DataFrame({
        'value': new_value_column,
        'text': new_text_column
    })
    return new_df

def transform_df(new_df, unique_identifier):
    new_df[unique_identifier] = new_df.apply(lambda x: x.text if x.value == unique_identifier else None, axis=1).ffill()
    new_df = new_df.pivot_table(index=unique_identifier, columns='value', values='text', aggfunc='first')
    return new_df

In [215]:
council = "Salford"
config = {
    "Bedford": {
        "gridlines": {
            (52, 121): "plate",
            (121, 220): "vrm",
            (220, 312): "make",
            (312, 418): "model",
            (418, 482): "licence_start",
            (482, 538): "licence_end"
        },
        "unique_identifier": "plate"
    },
    "BCP": {
        "gridlines": {
            (42, 76): "reference_number",
            (76, 212): "name",
            (212, 268): "vrm",
            (268, 311): "licence_number",
            (311, 364): "licence_start",
            (364, 412): "licence_end",
            (412, 528): "make",
            (528, 590): "licence_type",
        },
        "unique_identifier": "reference_number"
    },
    "Cheshire West": {
        "gridlines": {
            (52, 160): "make",
            (160, 478): "model",
            (478, 540): "vrm"
        },
        "unique_identifier": "vrm"
    },
    "East Northamptonshire": {
        "gridlines": {
            (55, 130): "vrm",
            (160, 240): "make",
            (240, 540): "model"
        },
        "unique_identifier": "vrm"
    },
    "Fylde": {
        "gridlines": {
            (18, 87): "vrm",
            (87, 216): "make",
            (216, 340): "model",
            (340, 460): "licence_start",
            (460, 540): "licence_end"
        },
        "unique_identifier": "vrm"
    },
    "Great Yarmouth": {
        "gridlines": {
            (52, 160): "vrm",
            (160, 238): "make",
            (238, 540): "model",
        },
        "unique_identifier": "vrm"
    },
    "Horsham": {
        "gridlines": {
            (52, 103): "reference_number",
            (103, 152): "licence_type",
            (152, 218): "name",
            (218, 266): "vrm",
            (266, 320): "make",
            (320, 386): "model",
            (386, 446): "licence_start",
            (446, 484): "licence_end",
            (484, 540): "plate",
        },
        "unique_identifier": "vrm"
    },
    "Kettering": {
        "gridlines": {
            (55, 102): "vrm",
            (148, 214): "make",
            (214, 540): "model"
        },
        "unique_identifier": "vrm"
    },
    "Maldon": {
        "gridlines": {
            (35, 142): "plate",
            (142, 258): "vrm",
            (258, 312): "make",
            (312, 445): "model",
            (445, 576): "colour",
            (576, 648): "seats",
            (648, 736): "licence_start",
            (736, 800): "licence_end",
        },
        "unique_identifier": "vrm"
    },
    "Milton Keynes": {
        "gridlines": {
            (256, 288): "plate",
            (288, 336): "vrm",
            (336, 406): "make",
            (406, 488): "model",
            (488, 742): "name",
            (742, 800): "licence_end"
        },
        "unique_identifier": "plate"
    },
    "North Lanarkshire": {
        "gridlines": {
            (52, 140): "vrm",
            (140, 210): "make",
            (210, 360): "model",
        },
        "unique_identifier": "vrm"
    },
    "Powys": {
        "gridlines": {
            (3, 38): "plate",
            (98, 222): "vrm",
            (222, 312): "make",
            (312, 404): "model",
            (404, 482): "licence_start",
        },
        "unique_identifier": "plate"
    },
    "Swansea": {
        "gridlines": {
            (52, 130): "vrm",
            (130, 240): "make"
        },
        "unique_identifier": "vrm"
    },
    "Wealden": {
        "gridlines": {
            (56, 98): "plate",
            (98, 136): "vrm",
            (136, 206): "make",
            (206, 272): "model",
            (272, 315): "colour",
            (315, 404): "name",
            (450, 488): "licence_start",
            (496, 540): "licence_end",
        },
        "unique_identifier": "vrm"
    },
    "West Lothian": {
        "gridlines": {
            (52, 184): "licence_type",
            (184, 295): "vrm",
            (295, 372): "make",
            (372, 440): "model",
        },
        "unique_identifier": "vrm"
    },
    "West Morland Barrow": {
        "gridlines": {
            (55, 118): "vrm",
            (118, 200): "make",
            (200, 300): "model",
        },
        "unique_identifier": "vrm"
    },
    "Argyll and Bute": {
        "gridlines": {
            (52, 158): "vrm",
            (158, 245): "make",
            (245, 400): "model",
        },
        "unique_identifier": "vrm"
    },
    "East Lothian": {
        "gridlines": {
            (77, 170): "vrm",
            (220, 312): "make",
            (312, 418): "model",
        },
        "unique_identifier": "vrm"
    },
    "Renfrewshire": {
        "gridlines": {
            (52, 138): "reference_number",
            (138, 248): "vrm",
            (248, 440): "make",
        },
        "unique_identifier": "vrm"
    },
    "Salford": {
        "gridlines": {
            (28, 78): "reference_number",
            (78, 136): "licence_type",
            (136, 198): "licence_start",
            (198, 250): "licence_end",
            (286, 392): "make",
            (392, 450): "vrm",
        },
        "unique_identifier": "reference_number"
    }
}

gridlines = config[council]["gridlines"]
unique_identifier = config[council]["unique_identifier"]

pdf_path = f'pdfs/tabular/{council}.pdf'
df = extract_pdf_text(pdf_path)


In [216]:
df.head(50)

,page,text,x0,y0,x1,y1
0,1,LICENSED,30.362000,31.719793,91.028944,43.718793
1,1,HACKNEY,94.364666,31.719793,153.687722,43.718793
2,1,AND,157.023444,31.719793,183.013278,43.718793
3,1,PRIVATE,186.349000,31.719793,238.352666,43.718793
4,1,HIRE,241.688388,31.719793,270.353999,43.718793
5,1,VEHICLES,273.689721,31.719793,333.696720,43.718793
6,1,LAST,565.107000,31.719793,596.436389,43.718793
7,1,REFRESHED,599.772111,31.719793,673.769944,43.718793
8,1,19/04/2024,677.139000,31.719793,737.181996,43.718793
9,1,AT,740.522722,31.719793,756.517389,43.718793


In [217]:
df = assign_intervals_and_values(df, gridlines)
df = process_consecutive_values(df, target_value=unique_identifier)
df_identifier = update_identifier_ranges(df, unique_identifier)
df_reduced = df[["text", "value"]].reset_index(drop=True)
new_df = concatenate_values(df_reduced)
new_df = transform_df(new_df, unique_identifier)

In [218]:
new_df

value,licence_end,licence_start,licence_type,make,reference_number,vrm
reference_number,,,,,,
110496,07/03/2025,28/02/2024,HACKNEY,LTI TX4,110496,T900 NYM
110564,09/04/2025,10/04/2024,HACKNEY,MERCEDES VITO,110564,SH59 KWJ
110635,13/03/2025,23/02/2024,HACKNEY,PEUGEOT E7,110635,SF10 AZA
110660,13/06/2024,14/06/2023,HACKNEY,LTI TX4,110660,MK10 SWJ
110684,01/05/2025,11/04/2024,HACKNEY,LTI TX4,110684,MK10 SWW
...,...,...,...,...,...,...
179091,16/10/2024,17/04/2024,PRIVATE,SKODA OCTAVIA SE,179091,BJ19 VSO
179148,10/10/2024,11/04/2024,PRIVATE,TOYOTA COROLLA,179148,YD69 XBA
LICENCE,EXPIRY,ISSUE DATE,LICENCE,VEHICLE TYPE,LICENCE,REG


In [219]:
new_df.to_csv('inspection.csv')